# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846879101615                   -0.70    4.8
  2   -7.852326147525       -2.26       -1.53    1.0
  3   -7.852646111531       -3.49       -2.52    3.0
  4   -7.852646676585       -6.25       -3.37    2.5
  5   -7.852646685757       -8.04       -4.76    1.2
  6   -7.852646686722       -9.02       -5.19    4.0
  7   -7.852646686730      -11.11       -6.18    1.5
  8   -7.852646686730      -12.55       -7.26    2.5
  9   -7.852646686730      -14.57       -7.52    2.0
 10   -7.852646686730   +  -14.75       -8.63    2.0
 11   -7.852646686730      -14.75       -9.64    2.2
 12   -7.852646686730   +  -15.05       -9.95    2.0
 13   -7.852646686730   +  -14.75      -10.74    1.5
 14   -7.852646686730      -14.75      -11.69    2.0
 15   -7.852646686730   +  -15.05      -12.29    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846845392153                   -0.70           4.8
  2   -7.852527263209       -2.25       -1.63   0.80    2.0
  3   -7.852637438335       -3.96       -2.71   0.80    1.0
  4   -7.852646478584       -5.04       -3.30   0.80    2.2
  5   -7.852646677692       -6.70       -4.16   0.80    1.2
  6   -7.852646686549       -8.05       -4.78   0.80    1.5
  7   -7.852646686717       -9.77       -5.64   0.80    1.5
  8   -7.852646686730      -10.91       -6.41   0.80    2.2
  9   -7.852646686730      -12.59       -7.58   0.80    1.8
 10   -7.852646686730      -14.05       -8.18   0.80    2.2
 11   -7.852646686730      -14.75       -9.27   0.80    1.5
 12   -7.852646686730   +    -Inf       -9.51   0.80    2.5
 13   -7.852646686730   +    -Inf       -9.79   0.80    1.0
 14   -7.852646686730   +  -14.45      -10.86   0.80    1.0
 15   -7.852646686730      -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.442933e+01     4.158441e+00
 * time: 0.5102031230926514
     1     1.190810e+00     1.993544e+00
 * time: 0.9082019329071045
     2    -1.574824e+00     2.258942e+00
 * time: 0.944382905960083
     3    -3.504314e+00     1.817344e+00
 * time: 0.9961481094360352
     4    -4.877634e+00     1.710773e+00
 * time: 1.037132978439331
     5    -6.483805e+00     1.404218e+00
 * time: 1.078913927078247
     6    -7.311805e+00     8.021307e-01
 * time: 1.1194839477539062
     7    -7.608617e+00     8.079301e-01
 * time: 1.1479980945587158
     8    -7.751461e+00     2.682307e-01
 * time: 1.1768651008605957
     9    -7.792524e+00     1.730960e-01
 * time: 1.2058861255645752
    10    -7.814265e+00     1.853800e-01
 * time: 1.2338049411773682
    11    -7.829606e+00     5.546431e-02
 * time: 1.2620251178741455
    12    -7.840207e+00     3.420393e-02
 * time: 1.2890629768371582
    13    -7.844285e+00     2.374787e-02
 * time: 1.3172569274902

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846747154004                   -0.70    4.8
  2   -7.852303307278       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686673                   -2.55
  2   -7.852646686730      -10.25       -5.77
  3   -7.852646686730   +    -Inf      -11.87
  4   -7.852646686730      -14.75      -15.50


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.4562080625403004e-13
|ρ_newton - ρ_scfv| = 7.481886868096052e-13
|ρ_newton - ρ_dm|   = 0.016860477678723597
